In [2]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [3]:
my $practice_input1 = qq/########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<
/;
my $practice_input2 = qq/##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
/;

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^


In [4]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my ($p) = $_[0] // $_; return $p->[0].','.$p->[1] }
sub uniqp { my %h; @h{map $_->[0].','.$_->[1], @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}

In [5]:
sub state_iterator { my ($state, $fun, @steps) = @_; map { $state = $fun->($state) } @steps }
sub flip_string { join "\n", reverse split "\n", $_[0] }
sub process_day15_part1 {
    my ($input) = @_;
    my ($input_map, $input_instructions) = map { chomp; $_ } split /\n\n/, $input;
    $input_instructions =~ s/\n//g;
    my @shape = shape parse_2d_map_array $input_map;
    my $row = "[\x00-\xff]{$shape[1]}";

    my @states =
        state_iterator $input_map, sub {
            my ($state) = @_;
            no warnings 'uninitialized';
            if ($_ eq '<') { $state =~ s/\.([a-z]*\@)/$1./ir; }
            elsif ($_ eq '>') { $state =~ s/(\@[a-z]*)\./.$1/ir; }
            elsif ($_ eq '^') { flip_string((flip_string $state) =~ s/\@($row)(([a-z])($row([a-z]$row)*))?\./.$1\@$4$3/ir); }
            elsif ($_ eq 'v') { $state =~ s/\@($row)(([a-z])($row([a-z]$row)*))?\./.$1\@$4$3/ir; }
            else { die "invalid instruction: $_" }
        },
        split '', $input_instructions;

    my $final_state = $states[-1];
    say "debug: final_state:\n$final_state";

        sum
        grep defined,
        flatten_nd
        map_nd_indexed sub { my ($arr, $p) = @_; my $v = $arr->[$p->[0]][$p->[1]]; $v eq 'O' ? $p->[0] * 100 + $p->[1] : undef },
        parse_2d_map_array $final_state;
    # parse_2d_map_array $input_map;
}
timer sub {
    say Dumper process_day15_part1($practice_input1);
    say Dumper process_day15_part1($practice_input2);
    confirm process_day15_part1($practice_input1), 2028;
    confirm process_day15_part1($practice_input2), 10092;
}

debug: final_state:
########
#....OO#
##.....#
#.....O#
#.#O@..#
#...O..#
#...O..#
########
$VAR1 = 2028;

debug: final_state:
##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
$VAR1 = 10092;

debug: final_state:
########
#....OO#
##.....#
#.....O#
#.#O@..#
#...O..#
#...O..#
########
confirmed: 2028
debug: final_state:
##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
confirmed: 10092
[time] 0 seconds


0

In [6]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/15/input');
    # say $input;
    $res = process_day15_part1($input);
    say Dumper $res;
    confirm $res, 1465152;
};

debug: final_state:
##################################################
#OOO.OO#.OO..#O.O...O.OOO...#O#O....OOO........OO#
#O...O...OOO#..#.......OOOO#O..O....OOO#O...OO.O.#
#O.......OO##.......O...OO.#O.......O#O.O.#....O##
#..........#O.......OO.............OOOO..#..OO.O.#
#...........OOO..#.#..O.........#OO...OOOOO.O.#.O#
#...........O#.......OO........O......O.OO.O#O...#
#.......#..O........O#OO..............OOOO.......#
#O........O.OOO........O.....OO...O....OO#...#...#
#O...#.#.....OO#.O.....O......#.....O...O.OO#O.OO#
#..........O##O.O...O.#O......O.O....#O#OOO..O...#
#...OO......OOO###..O.#.....#O.......O#.##O.#O...#
#....#O.....O#OO.O..O...O..OO#OOO.......OOOOOOO#O#
#......#......OOOOOOO##O..#OOOOOO.........OO..#O.#
#............O#OO.O#OOOO....OO#OOO............O.O#
#OO#O.O...##.O......OOOOO...#OO..OO...........OOO#
#OOOOO##OOO....O..##OO..O..OOOO.OOO.........O..#.#
#OO.##O.O.....#..OOO.....OOO.......#....O......OO#
##O...O.O.......OO........O.....#O....O.OO#.OO..##
#.OO#...O#.

2

In [62]:
say post_answer('2024/day/15/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 15 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [15]:
sub expand_map {
    my ($map) = @_;
    my %table = (
        '#' => '##',
        '.' => '..',
        '@' => '@.',
        'O' => '[]',
    );
    return $map =~ s/(.)/$table{$1}/gre;
}

sub process_day15_part2 {
    my ($input) = @_;
    my ($input_map, $input_instructions) = map { chomp; $_ } split /\n\n/, $input;
    $input_map = expand_map($input_map);
    $input_instructions =~ s/\n//g;
    # my @shape = shape parse_2d_map_array $input_map;
    # my $row = "[\x00-\xff]{$shape[1]}";
    my $obst = "[a-zA-Z\\[\\]]";

    sub collision_detection {
        my ($state) = @_;
        my @shape = shape parse_2d_map_array $state;
        
        my @lines = split "\n", $state;
        my ($line_number) = grep $lines[$_] =~ /\@/, 0 .. $#lines;
        my $init_pattern = $lines[$line_number] =~ s/[^\@]/\x00/gr;
        my $pattern = $init_pattern =~ s/[^\x00]/\xff/gr;
        # say "extr pattern: ", unpack "H*", $pattern;
        # say "line number: $line_number";

        # my $wall_pattern = $lines[$line_number+1] =~ s/[^#]/\x00/gr =~ s/#/\xff/gr;
        # my $collision_pattern = $wall_pattern & $pattern;
        my %incomplete_box_map = ( "[\x00" => '[]', "\x00]" => '[]' );
        my $box_iterator = 1;
        my $box_pattern = (($lines[$line_number+$box_iterator] =~ s/[^\[\]]/\x00/gr) & $pattern) =~ s/(\[\x00|\x00\])/$incomplete_box_map{$1}/gr;
        my $total_box_pattern = $init_pattern;
        while ($box_pattern =~ /[^\x00]/) {
            $total_box_pattern .= $box_pattern;
            $box_iterator++;
            $pattern = $box_pattern =~ s/[^\x00]/\xff/gr;
            $box_pattern = (($lines[$line_number+$box_iterator] =~ s/[^\[\]]/\x00/gr) & $pattern) =~ s/(\[\x00|\x00\])/$incomplete_box_map{$1}/gr;
        }
        my $wall_pattern = (join '', map $lines[$line_number+$_], 1 .. $box_iterator) =~ s/[^#]/\x00/gr;
        my $collision_pattern = $wall_pattern & ($total_box_pattern =~ s/[^\x00]/\xff/gr);
        
        # say "wall pattern: ", $wall_pattern =~ s/\x00/./gr;
        # say "coll pattern: ", $collision_pattern =~ s/\x00/./gr;
        # say "box  pattern: ", $total_box_pattern =~ s/\x00/./gr;

        if ($collision_pattern !~ /[^\x00]/) {
            my $real_pattern = (join '', @lines[map $line_number+$_, 0 .. $box_iterator]);
            my $cleaned_pattern = $real_pattern ^ $total_box_pattern;
            my $res_pattern = ((($real_pattern ^ $total_box_pattern) =~ s/\./\x00/gr) ^ ("\x00" x $shape[1] . $total_box_pattern)) =~ s/\x00/./gr;
            # $real_pattern ^=
            # say "can move!";
            # say "real pattern: ", $real_pattern;
            # say "clea pattern: ", $cleaned_pattern;
            # say "rez  pattern: ", $res_pattern;
            # say "row: $_" foreach map substr($res_pattern, $_*$shape[1], $shape[1]), 0 .. $box_iterator;
            @lines[map $line_number+$_, 0 .. $box_iterator] = map substr($res_pattern, $_*$shape[1], $shape[1]), 0 .. $box_iterator;
        }

        $state = join "\n", @lines;
        return $state;
    }

    my @states =
        state_iterator $input_map, sub {
            my ($state) = @_;
            no warnings 'uninitialized';
            # say "input: $_\n$state";
            if ($_ eq '<') { $state =~ s/\.($obst*\@)/$1./ir; }
            elsif ($_ eq '>') { $state =~ s/(\@$obst*)\./.$1/ir; }
            # elsif ($_ eq '^') { flip_string((flip_string $state) =~ s/\@($row)(($obst)($row($obst$row)*))?\./.$1\@$4$3/ir); }
            elsif ($_ eq '^') {
                flip_string collision_detection flip_string $state;
            }
            elsif ($_ eq 'v') {
                collision_detection $state;
                # $state =~ s/\@($row)(($obst)($row($obst$row)*))?\./.$1\@$4$3/ir;
            }
            else { die "invalid instruction: $_" }
        },
        split '', $input_instructions;

    # say foreach @states;
    my $final_state = $states[-1];
    say "debug: final_state:\n$final_state";

        sum
        grep defined,
        flatten_nd
        map_nd_indexed sub { my ($arr, $p) = @_; my $v = $arr->[$p->[0]][$p->[1]]; $v eq '[' ? $p->[0] * 100 + $p->[1] : undef },
        parse_2d_map_array $final_state;
    # parse_2d_map_array $input_map;
}
timer sub {
#     say Dumper process_day15_part2(qq/#######
# #...#.#
# #.....#
# #..OO@#
# #..O..#
# #.....#
# #######

# <vv<<^^<<^^/);
    # say Dumper process_day15_part2($practice_input1);
    say Dumper process_day15_part2($practice_input2);
    confirm process_day15_part2($practice_input1), 1751;
    confirm process_day15_part2($practice_input2), 9021;
}

debug: final_state:
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
$VAR1 = 9021;

debug: final_state:
################
##......[][]..##
####....[]....##
##......[]....##
##..##...[]...##
##....@.......##
##......[]....##
################
confirmed: 1751
debug: final_state:
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
confirmed: 9021
[time] 0 seconds


0

Warning: Subroutine expand_map redefined at reply input line 1.

Subroutine collision_detection redefined at reply input line 21.

Subroutine process_day15_part2 redefined at reply input line 12.


In [16]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/15/input');
    # say $input;
    $res = process_day15_part2($input);
    say Dumper $res;
    # confirm $res, 1465152;
};

debug: final_state:
####################################################################################################
##[][]......[]##..[]......##[].........[][].[][][]......##..##......[]....................[]..[][]##
##....[]..[]......[]....##....##........[].......[].[]##..[][]..[]........[]..##[]......[][]..[]..##
##........[]......[]..####.............[].............##..............[][]##[]..[]..##....[]..[]####
##[]..............[][]##[]............[]........[].[].[]....[][][]..[]..[][]......##....[].....[].##
##....[]......[].[][].............##..##.......[][]...[]..[]....##....[][][]..[][][]..[]....##..[]##
##........[][][]..........##..........[].......[][].....[]........[]..[]..[][]..[][]..[]##.....[].##
##....[]........##........................##...[][]...............[]....[][]..[]..[]............[]##
##......[]....[][]...........[]...[].........[].[].[].[]..........[]..[][][]......##......##......##
##........##..##......[]......##[][][].[]......[]...........##..[]..[].

17

In [18]:
say post_answer('2024/day/15/answer', 2, $res); # 1511259

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 15 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1